In [ ]:
!pip install -q transformers datasets peft accelerate evaluate torch bitsandbytes

In [ ]:
import pandas as pd
import numpy as np
import torch
import time
import json
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from peft import LoraConfig, get_peft_model
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
file_path = r"/content/Sentences_66Agree.txt"  # <-- Update path if needed

# Use cp1252 encoding
with open(file_path, 'r', encoding='cp1252') as f:
    lines = f.readlines()

sentences = []
labels = []

for line in lines:
    line = line.strip()
    if not line:
        continue
    if "@positive" in line:
        label = "positive"
        text = line.replace("@positive", "").strip()
    elif "@negative" in line:
        label = "negative"
        text = line.replace("@negative", "").strip()
    elif "@neutral" in line:
        label = "neutral"
        text = line.replace("@neutral", "").strip()
    else:
        continue  # skip malformed lines

    sentences.append(text)
    labels.append(label)

print(f"✅ Loaded {len(sentences)} sentences.")

# Map labels to integers
df = pd.DataFrame({"text": sentences, "label": labels})
df["label"] = df["label"].map({"negative": 0, "neutral": 1, "positive": 2})

print(df["label"].value_counts())

In [ ]:
# Split dataset
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["label"])
val_df, test_df = train_test_split(test_df, test_size=0.5, random_state=42, stratify=test_df["label"])

# Convert to HuggingFace Datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
# 2️⃣ Tokenization
# ======================================
model_name = "yiyanghkust/finbert-tone"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128)

train_dataset = train_dataset.map(preprocess, batched=True)
val_dataset = val_dataset.map(preprocess, batched=True)
test_dataset = test_dataset.map(preprocess, batched=True)

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

In [ ]:
base_model = AutoModelForSequenceClassification.from_pretrained(
    "yiyanghkust/finbert-tone",
    num_labels=3,
    ignore_mismatched_sizes=True
)


In [ ]:
lora_config = LoraConfig(
    r=8,                      # LoRA rank
    lora_alpha=32,            # Scaling factor
    target_modules=["query", "value"],  # Apply LoRA to attention layers
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_CLS"
)


model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

In [ ]:
# 4️⃣ Define metrics
# ======================================
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    f1_per_class = f1_score(labels, predictions, average=None)

    return {
        "accuracy": acc,
        "f1": f1,
        "f1_negative": f1_per_class[0],
        "f1_neutral": f1_per_class[1],
        "f1_positive": f1_per_class[2],
    }


In [ ]:
# 5️⃣ Training arguments
# ======================================
training_args = TrainingArguments(
    output_dir="./news_classifier_lora",
    learning_rate=5e-5,                 # slightly higher than full FT
    per_device_train_batch_size=32,     # same as full FT
    per_device_eval_batch_size=32,
    num_train_epochs=2,                 # same as full FT
    weight_decay=0.01,
    logging_dir="./logs_lora",
    logging_steps=50,
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    fp16=True,                          # same as full FT if using GPU
    report_to="none",
)


In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 7️⃣ Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

In [ ]:
# 9️⃣ Evaluate LoRA model
# ======================================
eval_results_lora = trainer.evaluate(test_dataset)

print("-" * 40)
for key, value in eval_results_lora.items():
    if key.startswith('eval_'):
        metric_name = key.replace('eval_', '').upper()
        print(f"{metric_name}: {value:.4f}")

predictions_lora = trainer.predict(test_dataset)
y_pred_lora = np.argmax(predictions_lora.predictions, axis=-1)
y_true = predictions_lora.label_ids

In [ ]:
# 🔟 Save LoRA adapter (tiny size)
# ======================================
print("\n💾 Saving LoRA model...")
model.save_pretrained("./news_classifier_lora")
tokenizer.save_pretrained("./news_classifier_lora")

In [ ]:
# 🔢 Reports
# ======================================
print("\n📊 Classification Report (LoRA):")
print(classification_report(y_true, y_pred_lora, target_names=["negative", "neutral", "positive"]))

# Confusion matrix
cm = confusion_matrix(y_true, y_pred_lora)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens',
            xticklabels=["negative", "neutral", "positive"],
            yticklabels=["negative", "neutral", "positive"])
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix - LoRA Fine-tuning")
plt.show()

In [ ]:
metrics_df_lora = pd.DataFrame(trainer.state.log_history)

# 2️⃣ Separate training and evaluation logs
train_loss_df = metrics_df_lora[metrics_df_lora["loss"].notnull()]
eval_loss_df = metrics_df_lora[metrics_df_lora["eval_loss"].notnull()]

# 3️⃣ Plot
plt.figure(figsize=(8, 5))
plt.plot(train_loss_df["epoch"], train_loss_df["loss"], 'o-', label="Training Loss")
plt.plot(eval_loss_df["epoch"], eval_loss_df["eval_loss"], 's-', label="Validation Loss")

plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training vs Validation Loss - LoRA")
plt.legend()
plt.grid(True)
plt.show()